In [1]:
import geowrangler.raster_zonal_stats as rzs
import geopandas as gpd
import pandas as pd

from pathlib import Path

/home/abbymoreno/miniconda3/envs/lacuna-fund2/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.11.1-CAPI-1.17.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


# Extract WorldPop

Population count and population density datasets will be sourced from [WorldPop](https://hub.worldpop.org/project/categories?id=3).

### Set-up directories and input parameters

In [7]:
# data files
DATA_DIR = Path("../../../data/")
WP_DIR = DATA_DIR / "02-raw" / "worldpop"
OUTPUT_DIR = DATA_DIR / "04-output"

# population count
POP_COUNT_2020 = WP_DIR / "unconstrained_phl_ppp_2020.tif"
# population density
POP_D_2020 = WP_DIR / "phl_pd_2020_1km.tif"

ADMIN_BOUNDS = DATA_DIR / "01-admin-bounds" / "renamed_target_admin_bounds.gpkg"

### Load AOI

In [8]:
aoi = gpd.read_file(ADMIN_BOUNDS, driver="GPKG")

### Utils

In [9]:
def extract_wp_raster_stats(wp_dataset, wp_year):
    wp_raster_stats = rzs.create_raster_zonal_stats(
        aoi,
        wp_dataset,
        aggregation=dict(
            func=["sum", "count", "mean", "median", "std", "min", "max"],
            column="population",
            output=[
                "wp_total",
                "samples",
                "wp_mean",
                "wp_median",
                "wp_stdev",
                "wp_min",
                "wp_max",
            ],
        ),
        extra_args=dict(nodata=-99999),
    )

    wp_raster_stats_df = wp_raster_stats.drop(columns=["geometry"])
    wp_raster_stats_df = pd.DataFrame(wp_raster_stats_df)
    wp_raster_stats_df["wp_year"] = wp_year
    wp_raster_stats_df.to_csv(OUTPUT_DIR / f"worldpop-popcount-{wp_year}.csv")

    return wp_raster_stats_df

## Generate raster zonal stats from WorldPop `TIFs`

First manually download the rasters from WorldPop. To get the values from the raster datasets, see [Raster Zonal Stats](https://geowrangler.web.app/raster_zonal_stats.html)  geowrangler documentation.

In [10]:
%%time
pop_count_stats_2020 = extract_wp_raster_stats(POP_COUNT_2020, 2020)
pop_count_stats_2020

CPU times: user 11.2 s, sys: 300 ms, total: 11.5 s
Wall time: 11.5 s


,region_name,region_code,province_name,province_code,city_name,city_code,barangay_name,barangay_psgc_code,wp_min,wp_max,wp_mean,samples,wp_total,wp_stdev,wp_median,wp_year
0,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Lomboy,PH015518016,10.708253,134.088531,19.414621,54,1048.389526,16.566447,16.313622,2020
1,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Tapuac,PH015518031,49.934814,171.413712,94.131187,85,8001.150879,23.749223,90.292122,2020
2,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Pantal,PH015518022,16.287024,842.690857,164.335341,190,31223.714844,209.994516,61.385437,2020
3,Region I,PH010000000,Pangasinan,PH015500000,Dagupan City,PH015518000,Barangay I (T. Bugallon),PH015518024,58.865456,158.769135,104.896740,21,2202.831543,25.539194,109.470459,2020
4,Region III,PH030000000,Nueva Ecija,PH034900000,Palayan City,PH034919000,Imelda Valley,PH034919017,0.835072,26.618620,5.960308,773,4607.318359,4.582612,4.241848,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
874,National Capital Region,PH130000000,"NCR, Second District",PH137400000,City of Mandaluyong,PH137401000,Namayan,PH137401018,125.150398,1452.383423,238.824514,43,10269.454102,230.109190,192.013565,2020
875,National Capital Region,PH130000000,"NCR, Second District",PH137400000,City of Mandaluyong,PH137401000,Plainview,PH137401022,97.603096,833.701477,308.359435,131,40395.085938,172.421888,248.975357,2020
876,National Capital Region,PH130000000,"NCR, Third District",PH137500000,City of Navotas,PH137503000,Navotas West,PH137503007,205.322052,457.391479,297.308695,6,1783.852173,79.680442,290.425232,2020
877,National Capital Region,PH130000000,"NCR, Third District",PH137500000,City of Navotas,PH137503000,Tanza,PH137503014,9.469161,572.421936,106.176457,318,33764.113281,104.138088,59.956230,2020
